# How to access Globus resources using the command line
> Jesus Fernandez / Instituto de Fisica de Cantabria (IFCA), CSIC-Universidad de Cantabria

## Install required software (run just once)

We will need the Globus CLI to interact with the server and Globus Connect Personal to setup your own end-point, in order to be able to place unattended batch requests.

### Install Globus CLI

This is available through conda (or mamba). Here we are creating a specific environment for Globus:

```bash
mamba create -n globus
mamba activate globus
mamba install globus-cli
```

This provides the `globus` command that we will use below.

### Install Globus Connect Personal

This is available from globus.org. Download the latest version and just unpack the tar file:

```bash
wget https://downloads.globus.org/globus-connect-personal/linux/stable/globusconnectpersonal-latest.tgz
tar xzf globusconnectpersonal-latest.tgz
cd globusconnectpersonal-3.1.6/
```

### Setup your Globus local server

At this point you need a Globus account. You can get one from https://www.globusid.org/create. Use these credentials when requested by the following command:

In [ ]:
globus login

Next, you can create a local endpoint for your computer. Below, it is named `my-local-endpoint`, but use a sensible name (e.g. your computer name) because this will appear on the globus searches:

In [ ]:
globus endpoint create --personal my-local-endpoint

This will return somthing like:
```
Message:     Endpoint created successfully
Endpoint ID: c8dbe48e-62d1-11ee-b819-9b7665fb6de1
Setup Key:   182335b6-28d4-4607-8628-451505ccc310
```

You can setup this endpoint using the Setup Key above:

In [ ]:
./globusconnectpersonal -setup 182335b6-28d4-4607-8628-451505ccc310

One last step is configuring the paths in your system that this endpoint will show to Globus. I recommend sharing just a given folder for this purpose. This path is setup using the file `~/.globusonline/lta/config-paths`

```bash
$ cat ~/.globusonline/lta/config-paths 
/path/to/my/globus/accessible/folder,0,1

```
We  are now ready to start our personal endpoint and start transfering data with any other globus endpoint.

## Start your local endpoint

First of all, we need to activate our personal endpoint end. It was already configured in our prious steps, which do not need to be repeated. Just issue:

In [ ]:
./globusconnect -start &

This function will be useful below to retrieve the Task ID from our transfer.

In [1]:
function return_task_id(){
  tee globus-transfer.log | awk -F: '/Task ID/ {print $2}'
}

: 1

## Log in and search
We can now log in, retrieve our Local ID and set the destination folder for our tranfer. This must be accessible to our local server (see above): 

In [ ]:
globus login
globus session show

localid=$(globus endpoint local-id)
destination="${localid}:/path/to/my/globus/accessible/folder/AND/MAYBE/SOME/SUBFOLDER"

We can now search for data:

In [4]:
globus endpoint search SAAG20YR

ID                                   | Owner                                                        | Display Name                             
------------------------------------ | ------------------------------------------------------------ | -----------------------------------------
5545b63c-c0e6-11ec-8f87-e31722b18688 | edefbccb-49c6-44bb-ba45-73fbb48d2834@clients.auth.globus.org | SAAG20YR HISTORICAL Single Variable Files
8fdfaa82-c0e6-11ec-8f87-e31722b18688 | edefbccb-49c6-44bb-ba45-73fbb48d2834@clients.auth.globus.org | SAAG20YR HISTORICAL wrf2d wrf3d          
(globus) 

: 1

Get the corresponding ID for a data set endpoint:

In [5]:
endpoint="5545b63c-c0e6-11ec-8f87-e31722b18688" # SAAG20YR HISTORICAL Single Variable Files
globus collection show $endpoint

Display Name:                SAAG20YR HISTORICAL Single Variable Files
Owner:                       kyoko@identity.7a577b.6fbd.data.globus.org
ID:                          5545b63c-c0e6-11ec-8f87-e31722b18688
Collection Type:             guest
Storage Gateway ID:          0dc590ef-281f-497a-9574-3a2236a8b933
Connector:                   POSIX
Disable Anonymous Writes:    False
High Assurance:              False
Authentication Timeout:      43200
Multi-factor Authentication: False
Manager URL:                 https://7a577b.6fbd.data.globus.org
HTTPS URL:                   https://g-1bd04a.7a577b.6fbd.data.globus.org
TLSFTP URL:                  tlsftp://g-1bd04a.7a577b.6fbd.data.globus.org:443
Force Encryption:            False
Public:                      True
Keywords:                    ['SAAG', 'WRF', 'NCAR']
(globus) 

: 1

And use `globus ls` to explore the contents. Here, we retrieve a file list, ready to be used in a batch transfer:

In [ ]:
origin="${endpoint}:HFX"
globus ls "${origin}" | sed 's,\(.*\),\1 \1,' > filelist.txt
cat filelist.txt

## Transfer data
This file lists the origin and destination names we'd like to use. We can use this file to initiate a transfer:

In [ ]:
# Launch transfer (batch)
taskid=$( globus transfer "${origin}" "${destination}" --label "Batch transfer" --batch filelist.txt | return_task_id )
cat globus-transfer.log

You can check the status of your transfer using:

In [ ]:
globus task show ${taskid}